In [1]:
# save & restore in tensorflow
import numpy as np
import tensorflow as tf

In [2]:
# specify save file name
save_file = './model.ckpt'

# create two tensor variable
number_of_inputs = 3
number_of_hidden_layer = 4
weights = tf.Variable( tf.truncated_normal([number_of_inputs, number_of_hidden_layer], stddev=0.1, mean=0) )
bias = tf.Variable( tf.zeros(number_of_hidden_layer) )

# initiate saver
saver = tf.train.Saver()

# start tensorflow session
with tf.Session() as sess:
    # initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # print variables
    print('Weights: ')
    print(sess.run(weights))
    print('Bias: ')
    print(sess.run(bias))
    
    # save model
    saver.save(sess, save_file)



Weights: 
[[ 0.11619949 -0.12168008  0.00361604 -0.09559458]
 [ 0.09213245  0.04759819 -0.00943307  0.15156533]
 [-0.03452    -0.00723098 -0.16533849  0.05939751]]
Bias: 
[ 0.  0.  0.  0.]


In [4]:
# remove previous variables
tf.reset_default_graph()

# Two Variables: weights and bias
weights = tf.Variable( tf.truncated_normal([number_of_inputs, number_of_hidden_layer]) )
bias = tf.Variable( tf.zeros(number_of_hidden_layer) )

# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

with tf.Session() as sess:
    # Load the weights and bias
    saver.restore(sess, save_file)

    # Show the values of weights and bias
    print('Weight:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

Weight:
[[ 0.11619949 -0.12168008  0.00361604 -0.09559458]
 [ 0.09213245  0.04759819 -0.00943307  0.15156533]
 [-0.03452    -0.00723098 -0.16533849  0.05939751]]
Bias:
[ 0.  0.  0.  0.]


In [5]:
# Save trained model
from tensorflow.examples.tutorials.mnist import input_data

In [7]:
tf.reset_default_graph()

learning_rate = 0.001
n_inputs = 784
n_classes = 10

# import MNIST data
mnist = input_data.read_data_sets('.', one_hot=True)

# x, y
features = tf.placeholder(tf.float32, [None, n_inputs])
labels = tf.placeholder(tf.float32, [None, n_classes])

# w, b
weights = tf.Variable(tf.truncated_normal([n_inputs, n_classes]))
bias = tf.Variable(tf.truncated_normal([n_classes]))

# logits
logits = tf.matmul(features, weights) + bias

# loss optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [9]:
import math

# start train model
save_file = './train_model.ckpt'
batch_size = 128
n_epoch = 100

saver = tf.train.Saver()

# start tf
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epoch):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)
        
        # loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(optimizer,
                    feed_dict={features: batch_features,
                              labels: batch_labels})
            
        # print status
        if epoch % 10 == 0:
            valid_accuracy = sess.run(accuracy,
                                      feed_dict={features: mnist.validation.images,
                                                 labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(epoch, valid_accuracy))
                
    
    # save the model
    saver.save(sess, save_file)
    print('Trained model saved!!')
    

Epoch 0   - Validation Accuracy: 0.12800000607967377
Epoch 10  - Validation Accuracy: 0.362199991941452
Epoch 20  - Validation Accuracy: 0.4927999973297119
Epoch 30  - Validation Accuracy: 0.5727999806404114
Epoch 40  - Validation Accuracy: 0.628600001335144
Epoch 50  - Validation Accuracy: 0.6654000282287598
Epoch 60  - Validation Accuracy: 0.6916000247001648
Epoch 70  - Validation Accuracy: 0.7092000246047974
Epoch 80  - Validation Accuracy: 0.723800003528595
Epoch 90  - Validation Accuracy: 0.7355999946594238
Trained model saved!!


In [10]:
# let's load saved model file again
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: mnist.test.images, labels: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

Test Accuracy: 0.7494000196456909
